# Lake_Bilancino
This is an artificial lake near Florence made with a dam on the river Sieve. It holds 69 km³ of water, is 31 metres at its deepest point and, covers an area of 5km². The task is to predict the Lake Level and Flow Rate for the next day from historical Rainfall, Temperature, Lake Level and Flow Rate data. 

## Our Understanding of the Lake Mechanics
The daily change in the lake water volume is the sum of the inflows less the sum of the outflows which for this model we assume:
1. The inflow is just from the rain runoff in the catchment area.   
2. The outflows are just evaporation and river outflows. 

This ignores losses due to seepage into the ground (likely to be small) or inflows from underground reserves.

### Rainfall
We are provided with the 24hr rainfall data for the catchment area. Not all this rainfall will end up 
in the dam, i.e.,some will seep into the ground, evaporate or be lost to other rivers/catchments. 
It is likely that it will take hours or days for the rainfall to arrive at the Lake and the inflow 
may be spread over several days.  We explore this below by looking at cross correlations between the 
daily lake level changes and the rainfall. We see correlations at days 0, -1, -2, and -3, with the 
peak correlation at day -1, i.e., the average lag between the rain falling and the inflow is 1 day.  
Since these Rainfall measurements are from adjacent areas it is likely they are highly correlated
and we explore this below. 

### Evaporation
The key factors affecting evaporation are
* Temperature: As the temperature increases, the rate of evaporation also increases. 
* Surface area: As the surface area increases, the rate of evaporation increases. 
* Humidity: The rate of evaporation decreases with an increase in humidity. 
* Wind speed: Increase in wind speed results in increased evaporation. 

The only data we are provided is the Temperature.  As the surface areas does not vary greatly on a daily basis this can safely be ignored. The importance of humidity and/or wind speed is unknown and may need to be investigated but is ignored in the following models.

### River Outflow
While we are given this it is considered to be an target (or output) variable. This means we can't use "tomorrows" river outflow when predicting "tomorrows" Lake Level. Instead we need to predict the River Outflow and use that. In the modeling below we find that simply using today's outflow in the model provides a prediction that is very close to using the actual measured outflow from "tomorrow". 

In many damns the outflow is manually controlled to manage the lake levels so that:
1. Below a specific level the outflow is a constant low amount, typically designed to maintain the health of the associated downstream water ecosystem.
2. Above a specific level the outflow increases and to maintain some optimal Lake Level range. 

This manual control may have significant impact on modeling the River Outflow. Specifically the relationship between Lake Level and River Outflow may inherently be nonlinear and, the relationship may suddenly and unpredictably change if the policy on how outflow is managed is changed. We see evidence for both in the analysis below.

### Lake Level
We assume the change in Lake Level is proportional to the change in lake volume (i.e., inflows - outflows). Since the lake has an irregular volumetric profile the change in level is not exactly proportional to volume, and may be dependant on the Lake Level, but this is a reasonable approximation for daily changes.

This means that input variables (i.e., Rainfall, Temperature) will be correlated with Lake Level Changes (though the change in volume) rather than with the Lake Level. This is because the Lake Level is the accumulation of all the prior Lake Level Changes.  This is clearly seen below in the correlation between Rainfall and Lake Level and Lake Level Changes and with autocorrelation plots.  


## Summary
Based on this understanding of the physical model, our hypothesis is the daily Lake Levels Changes are driven by the inflow of rain from the last few days less the outflows from evaporation and the flow rate and, the Lake Levels are calculated by adding this change to the current level. We use these insights into the physical model to drive the Exploratory Data Analysis, Feature Engineering and Modeling below.

In [ ]:
import os
import numpy as np
import pandas as pd
import missingno as msno
import sys

import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.rcParams['figure.figsize'] = (16, 4)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

from termcolor import colored

In [ ]:
def missing(df):
    return df.isnull().sum()/len(df)*100
    
def date_features(df):
    df['year'] =  df.index.year
    df['month'] =  df.index.month
    df['dayofyear'] =  df.index.dayofyear
    df['dayofweek'] =  df.index.dayofweek

    df['weekofyear'] =  df.index.weekofyear

In [ ]:
dfs = dict()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            name = filename[:-4]
            dfs[name] = pd.read_csv(os.path.join(dirname, filename), parse_dates=['Date'], dayfirst=True)
            

print("Names of objects and shape of datasets:")
for key in dfs:
    print(f"{key}, {dfs[key].shape}")

In [ ]:
dfs_time = pd.DataFrame(dfs.keys(), columns=['name'])
dfs_time['first_date'] = [dfs[key]['Date'].min() for key in dfs]
dfs_time['last_date'] = [dfs[key]['Date'].max() for key in dfs]
dfs_time['duration'] = (dfs_time['last_date'] - dfs_time['first_date']).apply(lambda x: x.days)+1
dfs_time['unique_days'] = [dfs[key]['Date'].nunique() for key in dfs]
dfs_time['missing_days'] = dfs_time['duration']-dfs_time['unique_days']
dfs_time['null_rows'] = [dfs[key].isnull().all(axis='columns').sum() for key in dfs]
dfs_time['null_date'] = [dfs[key]['Date'].isnull().sum() for key in dfs]
dfs_time['missing_data'] = [dfs[key].isnull().sum().sum() for key in dfs]

dfs_time.sort_values("missing_data")[['name', 'first_date', 'last_date', 'missing_days', 'null_rows', 'null_date', 'missing_data']]

In [ ]:
dfs['Water_Spring_Madonna_di_Canneto'].dropna(subset=['Date'], inplace=True)
for key in dfs:
    dfs[key].set_index('Date', drop=True, inplace=True, verify_integrity=True)

## Exploratory Data Analysis
The data includes: 
* Rainfall for 5 catchment areas
* Temperature for Le Croci, which is close to the lake
* Lake Level and output Flow Rate which are the targets

### Missing Data
The visualizations below show there is daily data from 2002-06-03 to 2020-06-30, however the 2002 and 2003 data is missing all Rainfall data. We will drop data before 2004.

In [ ]:
msno.matrix(dfs["Lake_Bilancino"], freq="Y", figsize=(16, 3), fontsize=10);

In [ ]:
pd.DataFrame({year: missing(dfs["Lake_Bilancino"][str(year)]) for year in range(2002, 2021)})

## Seasonality
The charts below show the lake level has seasonal variation, that is similar across the years, however it's not clear how this relates to the Total Rainfall or the Flow Rate. 

We note the Lake Level has a sharp rise in January that flattens out. This pattern is consistent with practice of using Flow Rate to constrain the level at some maximum level as noted above.

In [ ]:
dfs["Lake_Bilancino"][['Lake_Level', 'Flow_Rate','Temperature_Le_Croci','Rainfall_S_Piero','Rainfall_Mangona', 
                       'Rainfall_S_Agata', 'Rainfall_Cavallina', 'Rainfall_Le_Croci']].\
                        plot(title='Lake Bilancino Daily Data', subplots=True, figsize=(20, 10));

In [ ]:
dfs["Lake_Bilancino"][['Lake_Level', 'Flow_Rate','Temperature_Le_Croci','Rainfall_S_Piero','Rainfall_Mangona', 
                       'Rainfall_S_Agata', 'Rainfall_Cavallina', 'Rainfall_Le_Croci']].resample("M").mean().\
                        plot(title='Lake Bilancino Monthly Averages', subplots=True, figsize=(20, 10));

From the Augmented Dickey Fuller test we conclude that the Lake_Level is a stationary time series, which means the statistical properties (i.e., mean, variance, autocorrelation) are constant over time. The same is true of Flow Rate

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(dfs["Lake_Bilancino"].Lake_Level)
print('Lake Level ADF Statistic: {:.3f},p-value: {:.3f}'.format(result[0], result[1]))
from statsmodels.tsa.stattools import adfuller
result = adfuller(dfs["Lake_Bilancino"].Flow_Rate['2004':'2020'])
print('Flow Rate ADF Statistic: {:.3f},p-value: {:.3f}'.format(result[0], result[1]))

## Correlation
The correlation matrix below shows the 5 rainfall measurements are highly correlated (0.87 < r < 0.92) to each other, which may cause issues with Linear Regression models. One option is to sum these into a Total Rainfall, which has a similar correlation to Lake Level and Lake Level Change as the individual rainfalls.

In [ ]:
rainfall=['Rainfall_Le_Croci', 'Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata', 'Rainfall_Cavallina']
dfs["Lake_Bilancino"]['Total_Rainfall'] = dfs["Lake_Bilancino"][rainfall].sum(axis=1)
dfs["Lake_Bilancino"]['Lake_Level_Change'] = dfs["Lake_Bilancino"].Lake_Level.diff()

In [ ]:
col_order = ['Rainfall_Le_Croci', 'Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata', 'Rainfall_Cavallina', 'Total_Rainfall', 'Temperature_Le_Croci', 'Flow_Rate', 'Lake_Level', 'Lake_Level_Change']
dfs["Lake_Bilancino"][col_order].corr().style.background_gradient(cmap='coolwarm').set_precision(2)

With this understanding we eliminate the individual rainfall data from the correlation matrix to more clearly show the key relationships between input and output variable:
* Lake Level Change (r = 0.25) and Flow Rate (r = 0.17) are weakly correlated to Total Rainfall 
* Flow Rate is weakly correlated to Temperature (-0.2) 
* Lake Level Change and Flow Rate are weakly correlated (0.30).

In [ ]:
col_order = ['Total_Rainfall', 'Temperature_Le_Croci', 'Lake_Level_Change', 'Lake_Level', 'Flow_Rate']
dfs["Lake_Bilancino"][col_order].corr().style.background_gradient(cmap='coolwarm').set_precision(2)

### Conclusion
As expected by our physical model:
1. The 5 rainfall measurements are highly correlated and can be replaced by a single Total Rainfall
2. The Rainfall is not correlated to and Lake Level but has weak correlation to Lake Level Change 
2. The Flow Rate and Lake Level 

## Impact of flow rate
Based on proposed physical model of how Flow Rate is used to control the Lake Levels we hypothesise that below a specific Lake Level the Flow Rate is held at a low, constant value, and above that level the Flow Rate is controlled to hold the Lake Level near some optimal value. If this is correct then:
1. A plot of Lake levels vs Flow Rate will be flat in low mode, then have a sharp knee at the transition level, then rise sharply.
2. The correlation between Rainfall and Lake Level Change should be high in the low level mode, but weak in the high level mode.

The first chart below,  Lake Level vs Flow Rate, follows this pattern with some yearly variation.  When broken out by year this shows that prior to 2016 the strategy was to have a low flow up to a Lake Level of 252, then the Flow Rate increase rapidly to keep the Lake Level near that value. However, from 2016 onward the strategy seems to have changed and the flow was slow increased after a lake level of 250.

In [ ]:
dfs["Lake_Bilancino"]['year'] = dfs["Lake_Bilancino"].index.year
sns.scatterplot(data=dfs["Lake_Bilancino"],x='Lake_Level', y= 'Flow_Rate', hue="year")
g = sns.FacetGrid(data=dfs["Lake_Bilancino"],col_wrap=7, col="year", height=2)
g.map(sns.scatterplot, "Lake_Level", "Flow_Rate")
g.add_legend();

## Cross correlation and lag
The physical model suggests it will take time for the rainfall to reach the lake so we look at the cross correlation of the Lake Levels Change with Rainfall. This shows clear peaks at Day -1 with correlations ranging from Day 0 to -3, i.e., the rain takes 0 to 3 days to reach the lake. This suggest our model should include **lagged** Rainfall features.

We did not find any notable cross correlations with the other features.

In [ ]:
x=dfs["Lake_Bilancino"].Lake_Level_Change['2004':'2020']
y=dfs['Lake_Bilancino']['2004':'2020'].Total_Rainfall
plt.xcorr(x, y, maxlags=10);
plt.title("Lake_Level_Change vs Total_Rainfall Cross Correlation");
plt.xticks(np.arange(-10, 11, 1));

The following autocorrelation and partial correlation plots shows strong correlation of the Lake level with prior days, which from the physical model is to be expected. For time series with a large autocorrelation the common solution is to take the difference, i.e., Lake Level Change.  The autocorrelation plot for the Lake Level Change shows that this first difference largely eliminates the autocorrelation.

This is again an indication that our model should focus on predicting the Lake Level Change, then add that to the prior day's Lake Level.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig, axs = plt.subplots(2, 2)
plot_acf(x=dfs["Lake_Bilancino"].Lake_Level['2004':'2020'], lags=50, title="Lake Level Autocorrelation", ax=axs[0, 0]);
plot_acf(x=dfs["Lake_Bilancino"].Lake_Level_Change['2004':'2020'], lags=50, title="Lake Level Change Autocorrelation",ax=axs[0, 1]);

plot_pacf(x=dfs["Lake_Bilancino"].Lake_Level['2004':'2020'], lags=50, title="Lake Level Partial Autocorrelation", ax=axs[1, 0]);
plot_pacf(x=dfs["Lake_Bilancino"].Lake_Level_Change['2004':'2020'], lags=50, title="Lake Level Change Partial Autocorrelation", ax=axs[1,1]);
plt.tight_layout();

Similarly the Flow Rate has a strong autocorrelation with the prior day which is removed by taking the first difference. 

In [ ]:
fig, axs = plt.subplots(2, 2)

plot_acf(dfs["Lake_Bilancino"].Flow_Rate['2004':'2020'], lags=50, title="Flow Rate  Autocorrelation",  ax=axs[0, 0]);
plot_acf(dfs["Lake_Bilancino"].Flow_Rate.diff()['2004':'2020'], lags=50, title="Flow Rate first difference  Autocorrelation",  ax=axs[0, 1]);

plot_pacf(dfs["Lake_Bilancino"].Flow_Rate['2004':'2020'], lags=50, title="Flow Rate Partial Autocorrelation",  ax=axs[1, 0]);
plot_pacf(dfs["Lake_Bilancino"].Flow_Rate.diff()['2004':'2020'], lags=50, title="Flow Rate first difference Partial Autocorrelation",  ax=axs[1, 1]);
plt.tight_layout();

### Conclusion
The conclusion 
1. The predictive model should use Lake Level Change as the target
2. The Flow Rate is higly dependant on the prior day's flow rate. 

## Variables

The Temperature and Lake Level Changes (below) have a normal like distribution where as Total Rainfall is skewed and may result in a poor prediction for linear regression models.  We note there are not clear cut correlations between these Rain Fall, Lake Level Change and Temp.

In [ ]:
sns.pairplot(dfs["Lake_Bilancino"][['Total_Rainfall', 'Temperature_Le_Croci', 'Lake_Level_Change']]);

THe following charts show the strong correlation between the 5 rainfall measurements. 

In [ ]:
sns.pairplot(dfs["Lake_Bilancino"][['Rainfall_Le_Croci', 'Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata', 'Rainfall_Cavallina']]);

# Feature Engineering
Based on the the Exploratory Data Analysis we propose adding the following features
- Lake level Change
- Total Rainfall
- 4 days of Lagged Total Rainfall
- 1 day of Lagged Lake Level
- 1 day of Lagged Lake Level Changes

In [ ]:
for i in range(1, 4):
    dfs["Lake_Bilancino"]['Total_Rain_Fall-{}'.format(i)] = dfs["Lake_Bilancino"].Total_Rainfall.shift(i)

dfs["Lake_Bilancino"]['Flow_Rate-1']  = dfs["Lake_Bilancino"]['Flow_Rate'].shift() 
dfs["Lake_Bilancino"]['Lake_Level-1']  = dfs["Lake_Bilancino"]['Lake_Level'].shift()
dfs["Lake_Bilancino"]['Lake_Level_Change-1']  = dfs["Lake_Bilancino"]['Lake_Level_Change'].shift()

## Lake Level Modeling
In this section we will explore a number of models using the insights gathered so far. Specifically, 
* Discard data prior to 2004-01-02 due to missing data.
* Use Lake Level Change as the Target.
* Use Total Rainfall and drop the individual rainfall features.
* Incorporate and proposed lagged features.
* Hold back data from 2019 and 2020 for validation.

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn import metrics



def score(target, test, pred, ax=None, title = None): 
    mae = metrics.mean_absolute_error(test, pred)
    if ax == None:
        fig, ax = plt.subplots(1)
    pd.DataFrame({'Actual': test, 'Predicted': pred}).plot(ax=ax, title=title)
    ax.set_xlabel('Date')
    ax.set_ylabel(target)
    ax.legend([target, 'Predicted (MAE: {:.5f})'.format(mae)]);

    return mae

#target = 'Lake_Level_Change'
train = dfs["Lake_Bilancino"]['2004-01-02':'2018']
test  = dfs["Lake_Bilancino"]['2019':'2020']

### Base line model
We consider three base line models for lake level change
* The lake level does not change. MAE: 0.04850 
* The lake level change is the same as today. MAE: 0.04442

In [ ]:
score('Lake_Level_Change', test['Lake_Level_Change'], [0]*len(test), title="Zero Lake Change");
score('Lake_Level_Change', test['Lake_Level_Change'], test['Lake_Level_Change-1'], title="Yesterday's Lake Change");

## Linear Regression Models
In this section we explore a few linear models using different feature sets inorder to understand feature importance and discover better models.  

We start with the basic features set of Temperature; Total Rainfall; 1, 2 & 3 day lagged Rainfall; and yesterday's Flow Rate. These features cover the key drivers of the inflow/outflow in our physical model, with the lagged Flow_Rate being used as an estimate of the Flow Rate. Obviously we can't use the actual Flow Rate as it unknown at the time of the prediction, but surprisingly we found using lagged Flow Rate gave a lower MAE.

This simple model (below) reflects the structure of the Lake Level Changes, i.e., the timing of most peaks and troughs, however the magnitudes are incorrect. We can see that Flow_Rate-1 has the largest importance, followed by Total_Rain_Fall-1.

This model has an  MAE  of 0.04575, which is 3% higher (worse) than the baseline model that simply uses yesterday's Lake Level Changes (MAE of 0.04442).

In [ ]:
def featurePlot(features, model, ax):
    importance = model.coef_
    pd.DataFrame(data={'importance': importance}, index=features ).sort_values(["importance"]).plot.barh(ax=ax)
    

def linearModel(target, features, fromDate = '2004-01-02'):
    train = dfs["Lake_Bilancino"][fromDate : '2018']
    test  = dfs["Lake_Bilancino"]['2019':'2020']
    model = LinearRegression(normalize=True).fit(train[features], train[target])
    pred = model.predict(test[features])
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])
    score(target, test[target], pred, ax0)
    featurePlot(features, model, ax1)
    plt.tight_layout();

In [ ]:
base_features = ['Total_Rainfall', 'Total_Rain_Fall-1', 'Total_Rain_Fall-2', 'Total_Rain_Fall-3', 'Temperature_Le_Croci', 'Flow_Rate-1']
linearModel('Lake_Level_Change', base_features)

With some experimentation we found that adding lagg features for each of Flow_Rate, Lake_Level_Change, Lake_Level yielded the best mode with an MAE of 0.04141, which is 6.7% better than the baseline. The feature importance plost show that the prior Lake_Level_Change is the most important feature and Total Rainfall and and the 2 and 3 day lagged versions have no impact.

In [ ]:
linearModel('Lake_Level_Change', base_features + ['Lake_Level_Change-1', 'Lake_Level-1'])

## Less is more
In modeling sometimes less features or less (really better) data gives better results, especially for Linear Regression. We experimented and found that reducing the features to just 'Total_Rain_Fall-2', 'Temperature_Le_Croci', 'Flow_Rate-1', 'Lake_Level_Change-1', 'Lake_Level-1' (i.e., remove the other lagged rainfalls) yield a lower MAE of 0.03636. 

In [ ]:
linearModel('Lake_Level_Change', ['Total_Rain_Fall-2', 'Temperature_Le_Croci', 'Flow_Rate-1', 'Lake_Level_Change-1', 'Lake_Level-1'])

Finally, remembering that in the EDA we notice that the relationship between Flow Rate and Lake Level Changes was different after 2016, we eliminated training before that data and got a much lower MAE of 0.03392, which is 24% better than the baseline. 

We observe below that the predicted Lake Level Change follows the structure well, but again the size of the peak/troughs are wrong. From the feature importance chart see that the prior day's Lake Level Change is the most important driver.

In [ ]:
linearModel('Lake_Level_Change', ['Total_Rain_Fall-2', 'Temperature_Le_Croci', 'Flow_Rate-1', 'Lake_Level_Change-1', 'Lake_Level-1'], fromDate='2016')

## Conclusion
The best linear model has 
- Temperature, which in practice will be a forecast
- Yesterday's Flow_Rate, Lake_Level_Change and Lake_Level
- Day -2 Total Rainfall
- Removes training data prior to 2016

This model has a 24% lower MAE than the baseline.

A key concern is we only forecast the next day's results based on today's known numbers, i.e., one step forecast. It is likely that a multi-step forecast is required, which is possible, however as the model uses lagged data involving the targets, forecasting errors will compound. Furthermore, we would need to forecast rain falls and temperatures. 

## Non Linear model
We performed the same set of experiments using XGBoost, which is one of the leading tree based models. We that for the same features this returned better (lower) MAE and also benefited from rationalising the features to just 'Total_Rain_Fall-2', 'Temperature_Le_Croci', 'Flow_Rate-1', 'Lake_Level_Change-1', 'Lake_Level-1', while eliminating training date prior to 2016. As can be seen below this does as good a job of matching the timing of the peak/troughs and a better job of estimating the magnitude. As a result the MAE is 0.03046 with is 31% better than the baseline and 10% better than the best linear model

In [ ]:
import xgboost as xgb

def XGBModel(target, features, fromDate = '2004-01-02'):
    train = dfs["Lake_Bilancino"][fromDate : '2018']
    test  = dfs["Lake_Bilancino"]['2019':'2020']
    model = xgb.XGBRegressor().fit(train[features], train[target])
    pred = model.predict(test[features])
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])
    score(target, test[target], pred, ax0)
    xgb.plot_importance(model, grid=False, ax=ax1,  show_values=False)
    plt.tight_layout();

In [ ]:
XGBModel('Lake_Level_Change', ['Total_Rain_Fall-2', 'Temperature_Le_Croci', 'Flow_Rate-1', 'Lake_Level_Change-1', 'Lake_Level-1'], fromDate='2016')

# Flow Rate
After some experimenation with features and type of model we found XGB with just 'Total_Rain_Fall-2', 'Flow_Rate-1', 'Lake_Level-1' provided the lowest MAE (0.44292) however this is 14% higher than the baseline model of using yesterday's Flow Rate (MAE 0.38995). This explains why the Lake Level models using Flow_Rate-1 work as well as thouse using the actual Flow-Rate-1 

In [ ]:
score('Flow_Rate', test['Flow_Rate'], test['Flow_Rate-1'], title="Yesterday's Flow_Rate");

In [ ]:
XGBModel('Flow_Rate',['Total_Rain_Fall-2', 'Flow_Rate-1', 'Lake_Level-1'], fromDate='2016')

# Conclusion
The non-linear model (XBBoost) for Lake Level Chave make the best predictions with an MAE of 0.03046. This is 31% better than the baseline and 10% better than the best linear model. This model is a single step forecast, i.e., "tomrrows" Lake Level.

The modeling of he Flow Rate found that a simple baseline model of using yesterday's Flow Rate had the lowest MAE. 